In [1]:
from collections import deque

class FlowNetwork:
    def __init__(self, vertices):
        self.vertices = vertices
        self.capacity = {}
        self.graph = {v: [] for v in vertices}

    def add_edge(self, u, v, cap):
        # 정방향 간선 추가
        self.graph[u].append(v)
        # 역방향 간선 추가 (초기 용량 0, Residual Graph를 위해 필요)
        self.graph[v].append(u)

        self.capacity[(u, v)] = cap
        self.capacity[(v, u)] = 0  # 역방향 초기 용량은 0

    def bfs(self, s, t, parent):
        visited = {v: False for v in self.vertices}
        queue = deque([s])
        visited[s] = True
        parent[s] = -1

        while queue:
            u = queue.popleft()
            for v in self.graph[u]:
                # 방문하지 않았고, 잔여 용량이 남아있는 경우 이동 가능
                if not visited[v] and self.capacity[(u, v)] > 0:
                    queue.append(v)
                    visited[v] = True
                    parent[v] = u
                    if v == t:
                        return True
        return False

    def edmonds_karp(self, s, t):
        parent = {v: None for v in self.vertices}
        max_flow = 0

        # 증가 경로가 존재하는 동안 반복
        while self.bfs(s, t, parent):
            path_flow = float('inf')

            # 경로상의 최소 잔여 용량(bottleneck) 찾기
            v = t
            while v != s:
                u = parent[v]
                path_flow = min(path_flow, self.capacity[(u, v)])
                v = u

            # 유량 업데이트 (정방향 -, 역방향 +)
            max_flow += path_flow
            v = t
            while v != s:
                u = parent[v]
                self.capacity[(u, v)] -= path_flow
                self.capacity[(v, u)] += path_flow
                v = u

        return max_flow

    def get_min_cut(self, s):
        # 최대 유량을 구한 후의 잔여 그래프에서 s로부터 도달 가능한 노드 탐색
        visited = {v: False for v in self.vertices}
        queue = deque([s])
        visited[s] = True

        S_set = set()
        S_set.add(s)

        while queue:
            u = queue.popleft()
            for v in self.graph[u]:
                if not visited[v] and self.capacity[(u, v)] > 0:
                    visited[v] = True
                    queue.append(v)
                    S_set.add(v)

        T_set = set(self.vertices) - S_set
        return S_set, T_set

# 그래프 설정
vertices = ['s', 'v1', 'v2', 'v3', 'v4', 't']
fn = FlowNetwork(vertices)

# 간선 및 용량 추가
fn.add_edge('s', 'v1', 16)
fn.add_edge('s', 'v2', 13)
fn.add_edge('v1', 'v3', 12)
fn.add_edge('v2', 'v1', 4)
fn.add_edge('v2', 'v4', 14)
fn.add_edge('v3', 'v2', 9)
fn.add_edge('v3', 't', 20)
fn.add_edge('v4', 'v3', 7)
fn.add_edge('v4', 't', 4)

# 1. Maximum Flow 계산
max_flow_value = fn.edmonds_karp('s', 't')

# 2. Minimum Cut 계산
s_set, t_set = fn.get_min_cut('s')

# 결과 출력
print(f"Maximum flow : {max_flow_value}") # 출력 : Maximum flow : 23
print(f"Minimum cut : S = {{{', '.join(sorted(s_set))}}}, T = {{{', '.join(sorted(t_set))}}}") # 출력 : Minimum cut : S = {s, v1, v2, v4}, T = {t, v3}

Maximum flow : 23
Minimum cut : S = {s, v1, v2, v4}, T = {t, v3}
